# softmax和分类模型

## softmax回归的基本概念
1. 用来处理分类问题
2. 如果用神经网络来描绘softmax回归，同线性回归一样，是一个单层神经网络，输出层是一个全连接层。通过使用softmax运算符（softmax operator）将输出值变换成值为正且和为1的概率分布 
$$ \hat{y_1},\hat{y_2},\hat{y_3} = softmax(O_1,O_2,O_3) $$
其中，
$$ 
\hat{y_1} = \frac{exp(o_1)}{\sum_{i=1}^3exp(o_i)} ,
\hat{y_2} = \frac{exp(o_2)}{\sum_{i=1}^3exp(o_i)} ,
\hat{y_3} = \frac{exp(o_3)}{\sum_{i=1}^3exp(o_i)}
$$

softmax回归对样本 i 分类的矢量计算表达式为
$$ o^{(i)} = x^{(i)}W+B $$
$$ \hat{y}^{(i)} = softmax(O^{(i)}) $$

## 交叉熵损失函数
对于样本i，我们构造向量$ y^{(i)} $ ，使其第$ y^{(i)}  $（样本i类别的离散数值）个元素为1，其余为0。这样我们的训练目标可以设为使预测概率分布 $ \hat{y}^{(i)} $ 尽可能接近真实的标签概率分布$ y^{(i)}  $。

交叉熵（cross entropy）
$$ H(y^{(i)},\hat{y}^{(i)}) = - \sum_{j=1}^{1}y^{(i)}_jlog\hat{y}^{(i)}_j $$

假设训练数据集的样本数为，交叉熵损失函数定义为
$$ l(\Theta)= \frac{1}{n}\sum_{i=1}^{n} H(y^{(i)},\hat{y}^{(i)}) $$
如果每个样本只有一个标签，那么交叉熵损失可以简写成 
$ l(\Theta)= -\frac{1}{n}\sum_{i=1}^{n}log\hat{y}^{(i)}_{{y}^{(i)}} $


# Torch 知识

## torchvision
    是服务于PyTorch深度学习框架的，主要用来构建计算机视觉模型。torchvision主要由以下几部分构成：

. torchvision.datasets: 一些加载数据的函数及常用的数据集接口；
. torchvision.models: 包含常用的模型结构（含预训练模型），例如AlexNet、VGG、ResNet等；
. torchvision.transforms: 常用的图片变换，例如裁剪、旋转等；
. torchvision.utils: 其他的一些有用的方法。

## 对多维Tensor按维度操作

In [4]:
import torch
X = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(X.sum(dim=0, keepdim=True))  # dim为0，按照相同的列求和，并在结果中保留列特征
print(X.sum(dim=1, keepdim=True))  # dim为1，按照相同的行求和，并在结果中保留行特征
print(X.sum(dim=0, keepdim=False)) # dim为0，按照相同的列求和，不在结果中保留列特征
print(X.sum(dim=1, keepdim=False)) # dim为1，按照相同的行求和，不在结果中保留行特征

tensor([[5, 7, 9]])
tensor([[ 6],
        [15]])
tensor([5, 7, 9])
tensor([ 6, 15])


In [10]:
!pwd

/Users/katch/Documents/learn/DataWhale/动手深度学习/task1


In [13]:
#softmax的简洁实现
# 加载各种包或者模块
import torch
from torch import nn
from torch.nn import init
import torchvision.transforms as transforms
import numpy as np
import sys
sys.path.append("/Users/katch/Documents/learn/DataWhale/动手深度学习/input")
# import d2lzh1981 as d2l


In [ ]:
#获取数据
mnist_train = torchvision.datasets.FashionMNIST(root='/Users/katch/Documents/learn/DataWhale/动手深度学习/input/FashionMNIST2065', train=True, download=True, transform=transforms.ToTensor())
mnist_test = torchvision.datasets.FashionMNIST(root='/Users/katch/Documents/learn/DataWhale/动手深度学习/input/FashionMNIST2065', train=False, download=True, transform=transforms.ToTensor())

batch_size = 256
train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=num_workers)


In [ ]:
#定义网络模型
num_inputs = 784
num_outputs = 10

class LinearNet(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(num_inputs, num_outputs)
    def forward(self, x): # x 的形状: (batch, 1, 28, 28)
        y = self.linear(x.view(x.shape[0], -1))
        return y
    
# net = LinearNet(num_inputs, num_outputs)

class FlattenLayer(nn.Module):
    def __init__(self):
        super(FlattenLayer, self).__init__()
    def forward(self, x): # x 的形状: (batch, *, *, ...)
        return x.view(x.shape[0], -1)

from collections import OrderedDict
net = nn.Sequential(
        # FlattenLayer(),
        # LinearNet(num_inputs, num_outputs) 
        OrderedDict([
           ('flatten', FlattenLayer()),
           ('linear', nn.Linear(num_inputs, num_outputs))]) # 或者写成我们自己定义的 LinearNet(num_inputs, num_outputs) 也可以
        )


In [ ]:
#初始化模型参数
init.normal_(net.linear.weight, mean=0, std=0.01)
init.constant_(net.linear.bias, val=0)


In [ ]:
#定义损失函数
loss = nn.CrossEntropyLoss() # 下面是他的函数原型
# class torch.nn.CrossEntropyLoss(weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='mean')


In [ ]:
#定义优化函数
optimizer = torch.optim.SGD(net.parameters(), lr=0.1) # 下面是函数原型
# class torch.optim.SGD(params, lr=, momentum=0, dampening=0, weight_decay=0, nesterov=False)


In [ ]:
#训练
def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params=None, lr=None, optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            y_hat = net(X)
            l = loss(y_hat, y).sum()
            
            # 梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            
            l.backward()
            if optimizer is None:
                d2l.sgd(params, lr, batch_size)
            else:
                optimizer.step() 
            
            
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

        
train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None, None, optimizer)